In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [0]:
import numpy as np 
import pandas as pd 
import os

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, Input
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import model_from_json

In [0]:
data = pd.read_csv('/kaggle/input/nnfl-assignment-2/final_train.csv')

In [0]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [0]:
sw=stopwords.words('english')

In [0]:
def stop_lem(review):
  
  tokens=review.split()
  tokens=[token.lower() for token in tokens if token.isalpha() and token not in sw]
 
  lemmatizer=WordNetLemmatizer()
  for i in range(len(tokens)):
    tokens[i]=lemmatizer.lemmatize(tokens[i])
  return ' '.join(tokens)
    

In [0]:
data['desc']=data['desc'].apply(lambda x:stop_lem(x))

In [0]:
num_words = 50000
tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')
tokenizer.fit_on_texts(data['desc'].values)
X = tokenizer.texts_to_sequences(data['desc'].values)
#word_index = tokenizer.word_index
#print('Found %s unique tokens.' % len(word_index))

max_length_of_text = 400
X = pad_sequences(X, maxlen=max_length_of_text)

#print(word_index)

In [0]:

y = data['rating']
y=to_categorical(y)
X_train, y_train = X,y
print(X_train.shape,y_train.shape)

In [0]:
embed_dim = 200 
lstm_out = 100 
batch_size =1024

inputs = Input((max_length_of_text, ))
x = Embedding(num_words, embed_dim)(inputs)    
x = LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2, return_sequences=False)(x) # The LSTM transforms the vector sequence into a single vector of size lstm_out, containing information about the entire sequence
x = Dense(11,activation='softmax')(x)
model = Model(inputs, x)
print(model.summary())


In [0]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [0]:
model.fit(X_train, y_train, batch_size = batch_size, epochs = 15,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',patience=2,min_delta=0.0001)])

In [0]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("2017A7PS0139G.h5")